In [1]:
from datasets import load_dataset,Dataset
import torch
from transformers import AutoModel,AutoModelForCausalLM
from transformers import AutoTokenizer
from peft import LoraConfig, get_peft_model

# Настройка модели и токенизатора
name_of_model = "Lamapi/next-1b"
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=name_of_model)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=name_of_model)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

Cancellation requested; stopping current tasks.


KeyboardInterrupt: 

In [2]:
print('SUI')

SUI


In [38]:
import json
with open("REOSH_Sharapov_posts.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)

In [42]:
dataset = Dataset.from_list(raw_data)

In [43]:
dataset

Dataset({
    features: ['id', 'text'],
    num_rows: 1178
})

In [60]:
def tokenize_function(example):
    tokens = tokenizer(
    example["text"],
    truncation=True,
    padding="max_length",
    max_length=28
)
    # Для Causal LM labels = input_ids
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

In [61]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1178 [00:00<?, ? examples/s]

In [48]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj","v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
lora_model = get_peft_model(model, config)

In [62]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./lora_next1b",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    logging_steps=50,
    save_steps=500,
    save_total_limit=2,
    learning_rate=3e-4,
    fp16=True,  # если есть GPU/MPS
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_dataset
)

trainer.train()


Step,Training Loss
50,4.906228
100,3.390805
150,2.864965
200,2.941614
250,2.571158
300,2.664441
350,1.800953
400,2.061615
450,1.894608
500,2.008696


TrainOutput(global_step=1767, training_loss=1.8129204278890871, metrics={'train_runtime': 572.593, 'train_samples_per_second': 6.172, 'train_steps_per_second': 3.086, 'total_flos': 415234459293696.0, 'train_loss': 1.8129204278890871, 'epoch': 3.0})

In [72]:
lora_model.to(device)
model.to(device)

Gemma3ForCausalLM(
  (model): Gemma3TextModel(
    (embed_tokens): Gemma3TextScaledWordEmbedding(262144, 1152, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma3DecoderLayer(
        (self_attn): Gemma3Attention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=1152, out_features=1024, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=1152, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=1024, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): Linear(in_features=1152, out_features=256, bias=False)
          (v_proj): lora.Linear(
            (base_layer): 

In [78]:
prompt = 'Почему трамп пришел к власти?'
inputs = tokenizer(prompt, return_tensors="pt")
inputs = {k: v.to(device) for k, v in inputs.items()}

In [85]:
with torch.no_grad():
  outputs = lora_model.generate(
    **inputs,
    max_length=150,
    temperature=0.5,
    top_p=0.9,
    do_sample=True,
    num_return_sequences=1
)

In [86]:
tokenizer.decode(outputs,skip_special_tokens=True)

['Почему трамп пришел к власти? ЗДЕСЬ ДОМОРАБЛИТИ ПРОМОПАводки\nБаркинг в Конгрессе, наплыв долларов, долги и Конгресс как за словами, а не делами? В 2017 году был первый забасточный день\nИван Петлевич Третьяков, политолог и эксперт по американской политике, провел масштабный опрос среди жителей России, которые были собраны в 2017 году\nИван Третьяков, проведший опрос в 2017 году, обнаружил, что 65% россиян не верят в заявления политиков\nИ']